In [2]:
from numpy import *

#载入数据，返回列表组成的列表，使其很容易封装到矩阵中
def loadDataSet(fileName):
    dataMat = []
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = map(float,curLine)
        dataMat.append(fltLine)
    return dataMat

#欧式距离
def distEclud(vecA,vecB):
    return sqrt(sum(power(vecA-vecB,2)))
#随机质心
def randCent(dataSet,k):
    n = shape(dataSet)[1]
    centroids = mat(zeros((k,n)))
    for j in range(n):
        minJ = min(dataSet[:,j])
        rangeJ = float(max(dataSet[:,j]-minJ))
        centroids[:,j] = minJ + rangeJ*random.rand(k,1)
    return centroids
        

In [13]:
#K-均值聚类算法
def kMeans(dataSet,k,distMeas=distEclud,createCent=randCent):
    m = shape(dataSet)[0] #列表行数，数据集行数，即条数
    clusterAssment = mat(zeros((m,2)))
    centroids = createCent(dataSet,k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):
            minDist = inf
            minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j,:],dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI
                    minIndex = j
            #如果任一数据分配发生改变，则迭代重复
            if clusterAssment[i,0] != minIndex:
                clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist**2    #最近质心，及其距离平方
        #print(centroids)
        #更新质心位置
        for cent in range(k):
            '''.A把矩阵转换为array数组，用==生成布尔数组，但此布尔数组为列向量数组，
            因此用nonzero转换为布尔数组对应的列标，并取第一维，其为bool数组中True
            对应的列标的行向量数组
            '''
            ptsInClust = dataSet[nonzero(clusterAssment[:,0].A==cent)[0]]
            #mean,sum等函数的axis=0参数表示对第一维进行操作
            #即对每一行操作，即对某一列的每一行进行计算操作
            centroids[cent,:] = mean(ptsInClust,axis=0)
    return centroids,clusterAssment
    

In [4]:
dataList = [[1,2,3],[1,2,2],[3,4,5],[3,5,4],[2,3,3],[4,5,5]]
dataMat = mat(dataList)
myCentroids,clustAssing = kMeans(dataMat,2)
print(myCentroids)
print(clustAssing)

[[ 2.53039573  3.54492197  2.20319838]
 [ 3.10963204  3.17487982  2.93800196]]
[[ 1.33333333  2.33333333  2.66666667]
 [ 3.33333333  4.66666667  4.66666667]]
[[ 1.33333333  2.33333333  2.66666667]
 [ 3.33333333  4.66666667  4.66666667]]
[[ 0.          0.33333333]
 [ 0.          0.66666667]
 [ 1.          0.66666667]
 [ 1.          0.66666667]
 [ 0.          1.        ]
 [ 1.          0.66666667]]


In [91]:
from numpy import *
m = mean(mat([[1,2],[2,3],[3,4]]),axis=0)
m.tolist()

[[2.0, 3.0]]

一种用于度量聚类效果的指标是SSE(Sum of Squared Error,误差平方和)。  
1. 将具有最大SSE值得簇划分成两个簇，然后对这些点运行K-均值算法，其中K为2。  
2. 将某两个簇合并。  
    a.合并最近得质心。通过计算所有质心之间的距离，然后合并最近的两个点。  
    b.合并两个使得SSE增幅最小的质心。合并两个簇然后计算总SSE值，在所有可能的两个簇上重复，直到找到合并最佳的两个簇。



**** 二分K-均值算法 ****  
将所有点看成一个簇  
当簇数目小于k时  
$ ~ $&nbsp;&#160;对于每一个簇  
&emsp;&#8195;计算总误差  
&emsp;&#8195;在给定的簇上面进行K-均值聚类（k=2）  
&emsp;&#8195;计算将该簇一分为二之后的总误差  
&emsp;选择使得误差最小的那个簇进行划分操作 

In [47]:

def biKmeans(dataSet,k,distMeas=distEclud):
    m = shape(dataSet)[0]       #数据行数，即条数
    clusterAssment = mat(zeros((m,2)))  #数据归类，第一列表示归为第几类，第二列为距离平方，也称误差平方
    centroid0 = mean(dataSet,axis=0).tolist()[0] #将所有点看成一个簇，生成第一个质心坐标
    centList = [centroid0]  #质心列表，其中初始包含一个列表，即一个质心
    print(centList)
    for j in range(m):
        clusterAssment[j,1] = distMeas(mat(centroid0),dataSet[j,:])**2 #计算所有点到初始质心的误差平方
    while (len(centList) < k):
        lowestSSE = inf   #最低SSE赋值为float类型最大实体
        for i in range(len(centList)):  #对每一个簇
            ptsInCurrCluster = dataSet[nonzero(clusterAssment[:,0].A==i)[0],:]  #取出属于第i簇中的所有数据集
            #对当前簇进行K-均值聚类，令K等于2,并返回其质心和数据的分配
            centroidMat,splitClustAss = kMeans(ptsInCurrCluster,2,distMeas)  
            sseSplit = sum(splitClustAss[:,1])       #被二分的SSE
            sseNotSplit = sum(clusterAssment[nonzero(clusterAssment[:,0].A!=i)[0],1])  #未被二分的SSE
            print('sseSplit,and not Split: ',sseSplit,sseNotSplit)
            if (sseSplit + sseNotSplit) < lowestSSE:
                bestCentToSplit = i      #经比较后选择被分的簇号
                bestNewCents = centroidMat   #分开后的两个新簇的质心坐标列表
                bestClustAss = splitClustAss.copy()  #被分数据的分配矩阵,包括所属簇号和损失损失误差
                lowestSSE = sseSplit + sseNotSplit  #更新最低SSE
        bestClustAss[nonzero(bestClustAss[:,0].A==1)[0],0] = len(centList) #将局部分配后的簇号更新为全局簇号
        bestClustAss[nonzero(bestClustAss[:,0].A==0)[0],0] = bestCentToSplit #将局部分配后的簇号更新为全局簇号
        print('the bestCentToSplit is :',bestCentToSplit)
        print('the len of bestClustAss is :',len(bestClustAss))
        centList[bestCentToSplit] = bestNewCents[0,:].tolist()[0] #更新被分配簇的质心坐标为分开后的第一簇质心坐标
        centList.append(bestNewCents[1,:].tolist()[0]) #将分开后的第二簇质心坐标追加在列表末尾
        print(centList)
        clusterAssment[nonzero(clusterAssment[:,0].A==bestCentToSplit)[0],:]=bestClustAss #更新全局分配矩阵
    return mat(centList),clusterAssment
                
                

In [48]:
dataList = [[1,2,3],[1,2,2],[3,4,5],[3,5,4],[2,3,3],[4,5,5]]
dataMat = mat(dataList)
myCentroids,clustAssing = biKmeans(dataMat,3)
print(myCentroids)
print(clustAssing)

[[2.3333333333333335, 3.5, 3.6666666666666665]]
sseSplit,and not Split:  4.0 0.0
the bestCentToSplit is : 0
the len of bestClustAss is : 6
[[3.3333333333333335, 4.666666666666667, 4.666666666666667], [1.3333333333333333, 2.3333333333333335, 2.6666666666666665]]
sseSplit,and not Split:  1.0 2.0
sseSplit,and not Split:  1.0 2.0
the bestCentToSplit is : 0
the len of bestClustAss is : 3
[[3.0, 5.0, 4.0], [1.3333333333333333, 2.3333333333333335, 2.6666666666666665], [3.5, 4.5, 5.0]]
[[ 3.          5.          4.        ]
 [ 1.33333333  2.33333333  2.66666667]
 [ 3.5         4.5         5.        ]]
[[ 1.          0.33333333]
 [ 1.          0.66666667]
 [ 2.          0.5       ]
 [ 0.          0.        ]
 [ 1.          1.        ]
 [ 2.          0.5       ]]


In [40]:
from numpy import *
x = [1,2,3,4]
y = mat(x)
y[0,:].tolist()[0]

[1, 2, 3, 4]